In [1]:
import json
import pandas as np
import re 
import random
import sqlite3

with open('lookup1.json') as json_file: 
    data = json.load(json_file)

with open('uniquelookup.json') as json_file:
    data2 = json.load(json_file)


In [2]:
conn = sqlite3.connect('testQ.db')
c = conn.cursor()


In [3]:
print(data['Location Entity'])

['(Province)', '(Province), (Country)', '(Province) in (Country)', '(County), (State)', '(County) in (State)', 'in the state of (State)', '(County), (State Abbreviation)', '(State)', '(State Abbreviation)', '(State, Country)', '(Country)', 'In the country of (Country)', 'in the world/around the world', 'globally', 'county', 'state', 'country', 'province']


In [4]:
question_template = 'What (Location Entity) has the (Value Entity) number of (Bed Entity)?'
output = {}
question_template_id = 'db3q2'


In [7]:
def replaceBed(): 
    query = ""
    if bed == 'staffed beds': 
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_STAFFED_BEDS")
    elif bed == 'licensed beds': 
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_LICENSED_BEDS")
    elif bed == 'ICU beds': 
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_ICU_BEDS")
    return query

entities = ['Location Entity', 'Value Entity', 'Bed Entity']
count = 1
while count < 300: 
    output[count] = []
    bed = random.choice(data['Bed Entity'])
    location = ['county', 'state']
    loc = random.choice(location)
    val = random.choice(data['Value Entity'])
    if val.find("(x)") >= 0:
        order = random.randint(2,7)
        val = val.replace("(x)", str(order))
        if order == 2: 
            val = val.replace("th", "nd")
        if order == 3: 
            val = val.replace("th", "rd")
    else:
        order = 1
   
    if val.find("most") >= 0 or val.find("highest") >=0 or val.find("Highest") >=0:
        ascending = False
    else:
        ascending = True
    query = ""
    populated_entities = []
    sql_template = "Select * from (Select  County_Name/State_Name, SUM(NUM_Licensed/Staffed/ICU_Beds) as sb From db3 Group by County/State_Name Order by SUM(NUM_Licensed/Staffed/ICU_Beds) asc/desc) where sb > 0 limit X,1"
    if loc == 'county': 
        query = replaceBed()
        query = query.replace("County_Name/State_Name", "COUNTY_NAME, STATE_NAME")
        query = query.replace("County/State_Name", "COUNTY_NAME, STATE_NAME")
        if ascending == False:
            query = query.replace("asc/desc", "desc")
            query = query.replace("X", str(order-1))
        else: 
            query = query.replace("asc/desc", "asc")
            query = query.replace("X", str(order-1))
        real_question = question_template.replace("(Bed Entity)", bed)
        real_question = real_question.replace("(Value Entity)", val)
        real_question = real_question.replace("(Location Entity)", loc)
        populated_entities.append(loc)
        populated_entities.append(val)
        populated_entities.append(bed)
    else: 
        query = replaceBed()
        query = query.replace("County_Name/State_Name", "STATE_NAME")
        query = query.replace("County/State_Name", "STATE_NAME")
        if ascending == False:
            query = query.replace("asc/desc", "desc")
            query = query.replace("X", str(order-1))
        else: 
            query = query.replace("asc/desc", "asc")
            query = query.replace("X", str(order-1))
        real_question = question_template.replace("(Bed Entity)", bed)
        real_question = real_question.replace("(Value Entity)", val)
        real_question = real_question.replace("(Location Entity)", loc)
        populated_entities.append(loc)
        populated_entities.append(val)
        populated_entities.append(bed)
        
    c.execute(query)
    print(real_question)
    print(query)
    result = c.fetchall()
    print(result)
    if result[0][0] == None or len(result) == 0:
        continue
    else:
        output[count].append({'question_template' : question_template_id, 'question_template' : question_template, 'entities':entities, 'question' : real_question, 'populated_entities': populated_entities, 'query_template' : sql_template, 'query' : query, 'database' : 'database 3'})
    count = count + 1


What state has the most number of ICU beds?
Select * from (Select  STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_ICU_BEDS) desc) where sb > 0 limit 0,1
[('Texas', 8719)]
What county has the 3rd highest number of licensed beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_LICENSED_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_LICENSED_BEDS) desc) where sb > 0 limit 2,1
[('Harris', 'Texas', 15722)]
What state has the least number of licensed beds?
Select * from (Select  STATE_NAME, SUM(NUM_LICENSED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_LICENSED_BEDS) asc) where sb > 0 limit 0,1
[('Vermont', 1706)]
What state has the 4th lowest number of licensed beds?
Select * from (Select  STATE_NAME, SUM(NUM_LICENSED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_LICENSED_BEDS) asc) where sb > 0 limit 3,1
[(None, 2585)]
What county has the lowest number of ICU beds?
Select * from (Select  COUNTY_NAME,

[('Bastrop', 'Texas', 1)]
What state has the highest number of staffed beds?
Select * from (Select  STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_STAFFED_BEDS) desc) where sb > 0 limit 0,1
[('California', 78128)]
What state has the 2nd lowest number of staffed beds?
Select * from (Select  STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_STAFFED_BEDS) asc) where sb > 0 limit 1,1
[('Alaska', 1532)]
What state has the most number of staffed beds?
Select * from (Select  STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_STAFFED_BEDS) desc) where sb > 0 limit 0,1
[('California', 78128)]
What state has the most number of ICU beds?
Select * from (Select  STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_ICU_BEDS) desc) where sb > 0 limit 0,1
[('Texas', 8719)]
What state has the lowest number of ICU beds?
Select * from (Select  STATE_NAME, SUM(NUM_ICU_

What county has the 2nd lowest number of ICU beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_ICU_BEDS) asc) where sb > 0 limit 1,1
[('Benzie', 'Michigan', 1)]
What state has the 6th lowest number of staffed beds?
Select * from (Select  STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_STAFFED_BEDS) asc) where sb > 0 limit 5,1
[('South Dakota', 2896)]
What county has the 4th highest number of staffed beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_STAFFED_BEDS) desc) where sb > 0 limit 3,1
[('Maricopa', 'Arizona', 9796)]
What state has the highest number of staffed beds?
Select * from (Select  STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_STAFFED_BEDS) desc) where sb > 0 limit 0,1
[('California', 78128)]
What state has the lowest num

What county has the highest number of ICU beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_ICU_BEDS) desc) where sb > 0 limit 0,1
[('Los Angeles', 'California', 2352)]
What state has the highest number of ICU beds?
Select * from (Select  STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_ICU_BEDS) desc) where sb > 0 limit 0,1
[('Texas', 8719)]
What state has the highest number of staffed beds?
Select * from (Select  STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_STAFFED_BEDS) desc) where sb > 0 limit 0,1
[('California', 78128)]
What county has the lowest number of staffed beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_STAFFED_BEDS) asc) where sb > 0 limit 0,1
[('Crosby', 'Texas', 2)]
What county has the 5th highest number of licensed beds

[('California', 83560)]
What county has the least number of staffed beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_STAFFED_BEDS) asc) where sb > 0 limit 0,1
[('Crosby', 'Texas', 2)]
What state has the most number of staffed beds?
Select * from (Select  STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_STAFFED_BEDS) desc) where sb > 0 limit 0,1
[('California', 78128)]
What state has the 4th lowest number of ICU beds?
Select * from (Select  STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_ICU_BEDS) asc) where sb > 0 limit 3,1
[('Hawaii', 219)]
What county has the least number of ICU beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_ICU_BEDS) asc) where sb > 0 limit 0,1
[('Bastrop', 'Texas', 1)]
What county has the most number of licensed beds

What county has the most number of licensed beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_LICENSED_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_LICENSED_BEDS) desc) where sb > 0 limit 0,1
[('Los Angeles', 'California', 24753)]
What state has the most number of licensed beds?
Select * from (Select  STATE_NAME, SUM(NUM_LICENSED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_LICENSED_BEDS) desc) where sb > 0 limit 0,1
[('California', 83560)]
What county has the most number of licensed beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_LICENSED_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_LICENSED_BEDS) desc) where sb > 0 limit 0,1
[('Los Angeles', 'California', 24753)]
What state has the most number of licensed beds?
Select * from (Select  STATE_NAME, SUM(NUM_LICENSED_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_LICENSED_BEDS) desc) where sb > 0 limit 0,1
[('California', 83560)]
What cou

What state has the least number of ICU beds?
Select * from (Select  STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_ICU_BEDS) asc) where sb > 0 limit 0,1
[('Vermont', 151)]
What county has the lowest number of ICU beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_ICU_BEDS) asc) where sb > 0 limit 0,1
[('Bastrop', 'Texas', 1)]
What county has the lowest number of licensed beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_LICENSED_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_LICENSED_BEDS) asc) where sb > 0 limit 0,1
[('Crosby', 'Texas', 2)]
What county has the most number of staffed beds?
Select * from (Select  COUNTY_NAME, STATE_NAME, SUM(NUM_STAFFED_BEDS) as sb From db3 Group by COUNTY_NAME, STATE_NAME Order by SUM(NUM_STAFFED_BEDS) desc) where sb > 0 limit 0,1
[('Los Angeles', 'California', 23367)]
What state has the lowest num

In [8]:
output

{1: [{'question_template': 'What (Location Entity) has the (Value Entity) number of (Bed Entity)?',
   'entities': ['Location Entity', 'Value Entity', 'Bed Entity'],
   'question': 'What state has the most number of ICU beds?',
   'populated_entities': ['state', 'most', 'ICU beds'],
   'query_template': 'Select * from (Select  County_Name/State_Name, SUM(NUM_Licensed/Staffed/ICU_Beds) as sb From db3 Group by County/State_Name Order by SUM(NUM_Licensed/Staffed/ICU_Beds) asc/desc) where sb > 0 limit X,1',
   'query': 'Select * from (Select  STATE_NAME, SUM(NUM_ICU_BEDS) as sb From db3 Group by STATE_NAME Order by SUM(NUM_ICU_BEDS) desc) where sb > 0 limit 0,1',
   'database': 'database 3'}],
 2: [{'question_template': 'What (Location Entity) has the (Value Entity) number of (Bed Entity)?',
   'entities': ['Location Entity', 'Value Entity', 'Bed Entity'],
   'question': 'What county has the 3rd highest number of licensed beds?',
   'populated_entities': ['county', '3rd highest', 'licensed

In [9]:
with open('datafordb3q2.json','w') as outfile:
    json.dump(output,outfile)